### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [7]:
num_iterations = 1
empty_list = []
for i in range(2520, 2620):

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.2611676073285409
cusum = 0.16365942313960075
eta = 2.3044373921566064
hl_amp_ratio = 0.5729624753130786
kurtosis = -0.7760814138603229
n_points = 1179
period = 0.6994147965576177
period_SNR = 215.05688668659116
period_log10FAP = -158.26416998568703
period_uncertainty = 0.00020175428502572368
phase_cusum = 0.3296515675110143
phase_eta = 0.3633767774934365
phi21 = -2.200324471643495
phi31 = -4.10867976711141
quartile31 = 0.353053712739559
r21 = 0.45251908104310307
r31 = 0.2748349461766034
shapiro_w = 0.9625096321105957
skewness = -0.36743273150256345
slope_per10 = -0.04701336290403103
slope_per90 = 0.044848140641989824
stetson_k = 0.7497541425403682
weighted_mean = 15.252745955148237
weighted_std = 0.22270346766546678
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = NonVar
class_probability = 0.51


######### Lightcurve features #########

amplitude = 0.05891798113961548
cusum = 0.1572664482806294
eta = 2.4538146851101006
hl_amp_ratio = 0.9090961150806391
kurtosis = -1.1299563988993941
n_points = 1787
period = 0.6831989629812661
period_SNR = 100.6185062110648
period_log10FAP = -24.587047338545673
period_uncertainty = 0.0001948471390487705
phase_cusum = 0.14411072188300356
phase_eta = 2.478412804661504
phi21 = -2.100561242066277
phi31 = -1.286154117086539
quartile31 = 0.2525869647999368
r21 = 0.37816099065145314
r31 = 0.1597449034808162
shapiro_w = 0.9643999338150024
skewness = 0.04797815806901435
slope_per10 = -0.013169167424497273
slope_per90 = 0.01199741542805422
stetson_k = 0.820081831372068
weighted_mean = 13.328121039135013
weighted_std = 0.1459867032779962
Pulled 1 of 1


...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.95


######### Lightcurve features #########

amplitude = 0.24554881696041164
cusum = 0.15979037109881503
eta = 2.062009576060634
hl_amp_ratio = 0.9222509298937157
kurtosis = -1.5534018963487053
n_points = 1131
period = 0.3185702222309475
period_SNR = 230.44068462691848
period_log10FAP = -208.21305857437534
period_uncertainty = 4.185655337410332e-05
phase_cusum = 0.2711725426424415
phase_eta = 0.2568789993780392
phi21 = -1.4365690350963543
phi31 = 0.4003502891077175
quartile31 = 0.378464854624168
r21 = 0.1175298718289507
r31 = 0.07834045184315669
shapiro_w = 0.8971265554428101
skewness = -0.04480899834340088
slope_per10 = -0.03175401694805093
slope_per90 = 0.03592126073855594
stetson_k = 0.9026427877458397
weighted_mean = 13.367739252099339
weighted_std = 0.1732556714817428
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = DSCT
class_probability = 0.35


######### Lightcurve features #########

amplitude = 0.13919180013411162
cusum = 0.06829828807567884
eta = 2.7526077488171157
hl_amp_ratio = 0.7268482937474732
kurtosis = -1.1283133199311741
n_points = 770
period = 0.21958971259189983
period_SNR = 211.95845076559223
period_log10FAP = -142.62356148757692
period_uncertainty = 2.1715039200231878e-05
phase_cusum = 0.27665728588748717
phase_eta = 0.20036127426557007
phi21 = 2.7218212200258884
phi31 = 6.008996001894362
quartile31 = 0.18737593157757537
r21 = 0.1747006253811361
r31 = 0.04694163194291009
shapiro_w = 0.947265625
skewness = -0.2941927368165165
slope_per10 = -0.05469313871028301
slope_per90 = 0.05240624747638157
stetson_k = 0.8626916814615231
weighted_mean = 12.42354196898876
weighted_std = 0.09629344045850233
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.29125213693391805
cusum = 0.20353959352258608
eta = 2.045275898870331
hl_amp_ratio = 0.4323207991488062
kurtosis = -0.3479716851565602
n_points = 476
period = 0.561124158238894
period_SNR = 120.1988133256944
period_log10FAP = -59.47468595000867
period_uncertainty = 0.00010971456648511957
phase_cusum = 0.22879922018564994
phase_eta = 0.5842115148715983
phi21 = 0.6623140489791136
phi31 = -1.834452034572558
quartile31 = 0.36403182055546246
r21 = 0.3428234346695489
r31 = 0.1743030587879398
shapiro_w = 0.9408766031265259
skewness = -0.7065903618624703
slope_per10 = -0.0636552523197146
slope_per90 = 0.050347652201002666
stetson_k = 0.6840230637184441
weighted_mean = 15.218591543121127
weighted_std = 0.2669213679899752
